In [ ]:
# Importing the libraries

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

                                                 Creating a Sine Wave

In [3]:
# x = np.linspace(0, 50, 501) ---> Creating an array of numbers from 0 to 50 with 501 numbers getting to it.
# y = np.sin(x) ---> Making y the sine of x

# plt.plot(x,y) ---> This will create a sine wave from 0 to 50 using the above numbers

                                              Creating the Sine Dataframe

In [ ]:
# df = pd.DataFrame(data = y, index = x, columns = ['Sine']) ---> This is provided you have established the variables x and y

                                                   Train Test Split 

In [ ]:
# Step 1: Determine how much samples are used for testing. we will use 10%
# test_percent = 0.1

# Step 2: Making a test point to have 10% of the total sample size and rounding it off
#         This method can be used to apply on any other samples in the future
# test_point = np.round(len(df)*test_percent)

# Step 3: Just minusing the total length of dataframe with the test_point of 10%
# test_index = int(len(df) - test_point)

# Step 4: Creating the training split. We use integer location slicing
# train = df.iloc[:test_index] ---> This will take all the samples from the start to before the test_index

# Step 5: Creating the training split. We use integer location slicing
# test = df.iloc[test_index:] ---> This will take all the samples from test_index to end

                                                       Scaling

In [ ]:
# The scaling method will be done differently in RNN as they are going to feed output back into themselves
# Therefore, we will have to scale our label(y) as well.
# We will follow the rules of only fitting into our training dataset

# from sklearn.preprocessieng import MinMaxScaler

# scaler = MinMaxScaler()
# scaler.fit(train)

# scaled_train = scaler.transform(train)

# scaled_test = scaler.transform(test)

                                         Creating Batches of training data

In [ ]:
# from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# length = 2 ---> Length of the output sequence. Do note that the larger the length, the longer time it will take to predict
# batch_size = 1


# generator = TimeseriesGenerator(scaled_training_data, scaled_training_data, length = length, batch_size = batch_size)

# [Do note that the first parameter is the index which is x and the second parameter is the label which is y.] 
# [Since they are both present in the training data, we use it for both in this case]
# [The generator will generate batches for us]

# X,y = generator[0] ---> Returns both the x and y from the generator. This is the first batch generated.

# If we output X itself, it will have a 2 arrays if we set the length as 2.
# If we output Y itself, it will only have 1.
# What we are essentially telling the RNN is that given the length(parameter in generator) of the arrays in X, 
# predict the y.

                                               Creating the Model

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, SimpleRNN, LSTM

# n_features = 1 ---> Basically x trying to predict y. So number of features should be 1

# model = Sequential()

# model.add(SimpleRNN(50, input_shape=(length, n_features)))
# [The 50 is the number of neurons you want. in this case, since we have length of 50, we put 50.]
# [The input_shape is the length(50) by the number of features(1) which is essentially 50 rows and 1 column according to example]
# [You can also put in the numbers manually eg; input_shape(50,1)]

# model.add(Dense(1))

# model.compile(optimizer='adam', loss='mse')

# model.summary() ---> Shows the different layers

# model.fit_generator(generator, epochs=5)
# [The fit_generator is from the library. The generator is the variable we declared above]

                                                Loss Evaluation

In [ ]:
# losses = pd.DataFrame(model.history.history)

# losses.plot() ---> Plots the loss


#                                          Evaluate the Test Data
# Essentially what we are doing is taking the last 50 points from training set and predicting the first point 
# into the future in the test set.

# first_evaluate_batch = scaled_train[-length:] ---> Do note that the length was already declared above
#                                               ---> You put the -length before the : is to take the last 50.
#                                               ---> If you put the it like [:-length] it will take the everything except
#                                                    the last 50

# first_evaluate_batch= first_evaluate_batch.reshape((1,length, n_features))

# model.predict(first_evaluate_batch) ---> Outputs the prediction on the first point in the future.
# scaled_test[0] ---> Shows the very first point of the test set to see if the prediction is close.

                                             Creating the For-Loop

In [ ]:
# Do note that the purpose for creating this For-Loop is the predict the ALL tests results by taking the initial 50 datas from
# training dataset and then predicting the the next point after the training which is the first point of test and repeating  
# this steps whereby now we take 50(49 training and 1 predicted result) samples to predict the second point of test 
# and so on and forth.

# Step 1: Creating an empty array to store the predicted test results
# test_predictions = []

# Step 2: Taking the last 50 datasets from scaled training data as the first evaluation batch
# first_evaluation_batch = scaled_train[-length:]

# Step 3: Reshaping the batch to fit the For-Loop
# current_batch= first_evaluate_batch.reshape((1,length, n_features))

#============================================[DO NOT USE THIS CODE]==========================================================

# np.append(current_batch[:,1:,:],[[[99]]], axis=1) ---> This code is essentially saying to move the current_batch along 
#                                                        by one time step. We are using this for the for loop below.

#============================================================================================================================ 

# Step 4: Using the For-Loop to append the predicted results for test to test_predictions.
# for i in range(len(test)):

#    current_pred = model.predict(current_batch)[0]
    
#    test_predictions.append(current_pred)
    
#    current_batch = np.append(current_batch[:,1:,:],[[current_pred]], axis=1)

                                            Inverse the Scaled Data

In [ ]:
# true_predictions = scaler.inverse_transform(test_predictions)

                                Plotting the Test data against the Predictions

In [ ]:
# test['Predictions'] = true_predictions ---> Creating another column under the test variable which was established from train
#                                             test split earlier to map the Predicted values by the model for comparison

# test.plot() ---> Plotting the Actual tests against the predicted tests.

                                           Include Early Stopping

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping

# early_stop = EarlyStopping(monitor='val_loss', patience = 2)


#==========================================Only if we include Validation Data==================================================

# If we create validation set, the length MUST be at least ONE shorter than the test data

# [We have to redefine the length as it can't be the same size as the scaled test data]
# length = (length of the test data) - 1

# generator = TimeseriesGenerator(scaled_train, scaled_train, length = length, batch_size=1)

# validation_generator = TimeseriesGenerator(scaled_test, scaled_test, length=length, batch_size = 1)
# [Do note that the length was already established as 50 from the example above]

#===============================================================================================================================

# model = Sequential()

# model.add(LSTM(50, input_shape=(length, n_features)))
#[Do note that the RNN has been replaced with LSTM]

# model.add(Dense(1))

# model.compile(optimizer='adam', loss='mse')

# model.summary() ---> Shows the different layers

# model.fit_generator(generator, epochs=20, validation_data=validation_generator, callbacks = [early_stop])
# [The fit_generator is from the library. The generator is the variable we declared above]


# Afterwards, you can proceed to do the For-Loop above with the exact same steps to do evaluate the test predictions
# with the plotting etc

                                 Forecasting into the future[Scaling ALL the Data]

In [ ]:
# We are essentially retraining all of the data
# Therefore, we will be scaling all of the data

# full_scaler = MinMaxScaler()
# scaled_full_data = full_scaler.fit_transform(df) ---> Scaling all of the data

                             Forecasting into the future[Creating and fitting the model]

In [ ]:
# generator = TimeseriesGenerator(scaled_full_data, scaled_full_data, length = length, batch_size = batch_size)

# model = Sequential()

# model.add(LSTM(50, input_shape=(length, n_features)))
# [Do note that the RNN has been replaced with LSTM]
# [If we are satisfied with the LSTM model instead of RNN, we use LSTM]

# model.add(Dense(1))

# model.compile(optimizer='adam', loss='mse')

# model.fit_generator(generator, epochs = 6)

                               Forecasting into the future[Forecast]

In [ ]:
# forecast = []
# first_evaluate_batch = scaled_train[-length:]
# current_batch= first_evaluate_batch.reshape((1,length, n_features))

# for i in range(25):

#    current_pred = model.predict(current_batch)[0]
    
#    forecast.append(current_pred)
    
#    current_batch = np.append(current_batch[:,1:,:],[[current_pred]], axis=1)

# [Do note that instead of putting len(test) in the for loop, we get to decide how many spaces we are 
# going to forecast into the future. We use 25 spaces in the example above]



# forecast = scaler.inverse_transform(forecast) ---> Inverting back the forecast

# forecast_index = np.arange(50.1,52.6, step=0.1) ---> Since for the above example, our index stops at 50 and we have forecasted
#                                                      25 steps into the future, this code mainly creates a new index starting 
#                                                      50.1 all through 52.6 for the forecasted figures to fit into it
#                                                      to be plotted as shown in the code below.

# plt.plot(df.index,df['Sine'])     ---> These 2 codes are to plot both the datas into a single table.
# plt.plot(forecast_index,forecast)      Using this Sine example, we are essentially creating another new Sine wave 25 points 
#                                        into the future.